In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('rule_dataset.csv')

def clean_braces(x):
    x = x.replace("()", "").replace(",", "").replace(" ", "")
    return str(x)
def add_braces(x):
    x = clean_braces(x)
    x = x+"()"
    return x

df['Syscall'] = df['Syscall'].apply(add_braces) #clean_braces
df.head(3)

# df['Syscall'] = df['Syscall'].apply(add_braces)
# df.to_csv("rule_dataset.csv", index=False)

,EntityType,ActionType,Source,Syscall,EnVerb,Sentence
0,FILE,FILE_READ,man,read(),read,read from a file descriptor
1,FILE,FILE_READ,man,readlink(),read,read value of a symbolic link
2,FILE,FILE_READ,man,readlink(),place,readlink() places the contents of the symbolic...


- 印出各類別的個數和內容，共 7 種 EntityType: FILE, PROC, DEVICE, ID, TIME, INFO, NET

In [3]:
df.iloc[0]

EntityType                           FILE
ActionType                      FILE_READ
Source                                man
Syscall                            read()
EnVerb                               read
Sentence      read from a file descriptor
Name: 0, dtype: object

In [4]:
printDetail = False #True
entityType_lst = df['EntityType'].unique()
lenSyscall = len(df['Syscall'].unique())
lenEnVerb = len(df['EnVerb'].unique())
print(f"Total rules: {len(df)}, EntityType: {df['EntityType'].unique()}")
print(f"For total {lenSyscall:2} uq syscalls & {lenEnVerb:2} uq verbs\n")

for entityType in entityType_lst:
    dataFileType = df[df['EntityType'] == entityType]
    lenEnTp = len(dataFileType)
    lenSyscall = len(dataFileType['Syscall'].unique())
    lenEnVerb = len(dataFileType['EnVerb'].unique())
    actionType_lst = dataFileType['ActionType'].unique()
    lenEnVerb_manpage = len(dataFileType[dataFileType['Source'] == 'man']['EnVerb'].unique())
    lenEnVerb_report = lenEnVerb - lenEnVerb_manpage

    # print(dataFileType.index, len(dataFileType.index))

    print(f"EntityType: {entityType:<6} has {len(dataFileType):2} rules | {lenSyscall:2} uq_syscalls | {lenEnVerb:2} uq_verbs | {len(actionType_lst):2} actionType\
          \n\t=>syscall: {list(dataFileType['Syscall'].unique())}\
          \n\t=>verb:    {list(dataFileType['EnVerb'].unique())}\
          \n\t=>verb:    from manpage {lenEnVerb_manpage}, from report {lenEnVerb_report}")

    for actionType in actionType_lst:
        dataActionType = dataFileType[dataFileType['ActionType'] == actionType]
        lenSyscall = len(dataActionType['Syscall'].unique())
        lenEnVerb = len(dataActionType['EnVerb'].unique())

        if printDetail:
            print(f"\t{actionType:<12} has {lenSyscall:2} uq syscalls & {lenEnVerb:2} uq verbs\
    {list(dataActionType['Syscall'].unique())} {list(dataActionType['EnVerb'].unique())}")
        else:
            print(f"\t{actionType:<12} has {lenSyscall:2} uq syscalls & {lenEnVerb:2} uq verbs")

    print()


Total rules: 240, EntityType: ['FILE' 'PROC' 'DEVICE' 'ID' 'TIME' 'INFO' 'NET']
For total 93 uq syscalls & 72 uq verbs

EntityType: FILE   has 104 rules | 42 uq_syscalls | 38 uq_verbs | 12 actionType          
	=>syscall: ['read()', 'readlink()', 'access()', 'statfs()', 'stat()', 'stat64()', 'fstat()', 'newfstatat()', 'lstat()', 'getcwd()', 'getdents()', 'poll()', 'ppoll()', 'dup()', 'dup2()', 'fcntl()', 'fcntl64()', 'mkdir()', 'open()', 'openat()', 'mmap()', 'mmap2()', 'umask()', 'link()', 'linkat()', 'symlinkat()', 'symlink()', 'fchdir()', 'chdir()', 'write()', 'rename()', 'fchmod()', 'chmod()', 'lchmod()', 'rm()', 'rmdir()', 'munmap()', 'unlink()', 'unlinkat()', 'remove()', 'close()', 'lseek()']          
	=>verb:    ['read', 'place', 'gather', 'fetch', 'look', 'see', 'use', 'extract', 'check', 'access', 'get', 'wait', 'duplicate', 'create', 'share', 'manipulate', 'find', 'set', 'create ', 'open', 'map', 'make', 'add', 'drop', 'mount', 'change', 'write', 'rename', 'update', 'format'

### Read Log

In [4]:
with open("./TestZone/Bert_alt_verb.log", 'r', encoding='utf-8') as f:
    lines = f.readlines()

convertion_list = []
exist_set = set() # 不記錄重複的 verb_origin, verb_alt，即便他們在不同句子的 cosine 不太相同
for line in lines:
    if not line.startswith("INFO:root:"):
        continue
    line = line.replace("INFO:root:", "")
    if line.startswith(" sentence is too long"):
        continue
    s = line.split('=>')
    verb_origin = s[0].strip()
    verb_alt_cos_sent = s[1]
    s = verb_alt_cos_sent.split('|')
    verb_alt = s[0].strip()
    cos = s[1].strip()
    sent = s[2].strip()
    if (verb_origin, verb_alt) in exist_set:
        continue
    convertion_list.append({
        "verb_origin": verb_origin,
        "verb_altern": verb_alt,
        "cosine": cos,
        "sentence": sent,
    })
    exist_set.add((verb_origin, verb_alt))

convertion_df = pd.DataFrame(convertion_list)
convertion_df = convertion_df.sort_values(by=['cosine'], ascending=False)
convertion_df

,verb_origin,verb_altern,cosine,sentence
29,modify,modify,cos: 0.9792,trend micro researchers also discovered that t...
33,add,add,cos: 0.9717,persistence persistence is achieved by the mal...
35,gather,gather,cos: 0.9699,this backdoor gathers the following data: cpu...
0,retrieve,create,cos: 0.8269,retrieve data from a file or directory
1,extract,extract,cos: 0.8255,table 2: data collected by the malware and sen...
23,drop,drop,cos: 0.8050,figure 11. content of the init script dropped ...
5,create,create,cos: 0.7918,"if it’s not running from these locations, then..."
16,use,create,cos: 0.7617,xorddos uses /proc/rs_dev character device to ...
12,pass,drop,cos: 0.7398,cron scriptthe malware creates a cron script a...
4,read,look,cos: 0.7303,it performs the following tasks:reads the cont...


In [14]:
# save the result table
convertion_df.to_csv("./TestZone/Bert_alt_verb_stat.csv")

### 動詞過濾 除去非動詞的 action word
- https://spacy.io/usage/linguistic-features
- https://machinelearningknowledge.ai/tutorial-on-spacy-part-of-speech-pos-tagging/
- 由於 pos 標籤一次只能貼上一個，所以有些詞會被誤標成名詞，如 name, hook, format。反而有些不該是動詞的單字被標籤上動詞，如 resolv.conf, m6_6n3。
- 經由這段測試，推薦自建黑名單剔除誤判成動詞的單字。

In [8]:
import spacy

oov_verbs = convertion_df['verb_origin']
none_verb_lst = ["resolv.conf", "wget", "m6_6n3", "se"] # wget 可以接受

nlp = spacy.load("en_core_web_sm")
for v in oov_verbs:
    doc = nlp(v)
    for token in doc:
        if token.pos_ != 'VERB':
            print(token.text, token.pos_, token.tag_) # ex: modify VERB VB
        # print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
        #         token.shape_, token.is_alpha, token.is_stop)

print("======")
for v in none_verb_lst:
    doc = nlp(v)
    for token in doc:
        print(token.text, token.pos_, token.tag_)


hardcode NOUN NN
format NOUN NN
name NOUN NN
se X FW
hook NOUN NN
se X FW
resolv.conf VERB VB
wget VERB VB
m6_6n3 VERB VB
se X FW


### 查看哪些 Syscall 未包含在 Rule 中

In [26]:
# Load Malware Sample ASG
# import re
import os
# from os import listdir
# from os.path import isfile, join
# from pathlib import Path
# import pandas as pd
# import pickle
from IPython.display import display

''' 自己寫的模組 '''
from Exp4_autore.ASG import AttackGraph, Node, Edge, FileTable #, build

family = 'GafGyt' # Darlloz
samplename = '933e9eff97f50b196b2bb0fef499640e' # dfeb77cb0ba28ac3ba4be55d7bc91fad
trace_path = f"../C ASG/trace/{family}/{samplename}.bin"
print("path is correct:", os.path.exists(trace_path))
asg = AttackGraph(trace_path)
asg.create()

path is correct: True


In [30]:
syscalls_in_asg = {step[2] for step in asg.step_list}
print(list(syscalls_in_asg)[:3], 'num of unique syscalls:', len(syscalls_in_asg))

['open', 'kill', 'fork'] num of unique syscalls: 15


In [31]:
tmp = df
tmp['Syscall'] = df['Syscall'].apply(clean_braces)
syscalls_in_rule = tmp['Syscall'].unique()
for asg_sy in syscalls_in_asg:
    if asg_sy not in syscalls_in_rule:
        print(asg_sy)

### 試寫 rule
- input: object 和 entityType
- output: 對應的 syscall (1或多個)
- Todo:
    1. 寫規則
    2. 回OPser導出資料
    3. 測試規則
    4. 擴充資料集 (source=report & expend 同義詞)
    5. extract BERT word vector

In [ ]:
df['EntityType'].unique()

In [ ]:
class OperationEvaluator:
    def __init__(self) -> None:
        self.df = self.read_rule()
        self.update_syscall_list = list(self.df[self.df['ActionType'].isin(['UPDATE'])]['Syscall'].unique())
        pass

    def read_rule(self) -> pd.DataFrame:
        df = pd.read_csv('rule_dataset.csv')

        def clean_braces(x):
            x = x.replace("()", "").replace(",", "").replace(" ", "")
            return str(x)

        df['Syscall'] = df['Syscall'].apply(clean_braces)
        return df

    def translate_rule(self, entityType: str, enVerb: str, object: str=None) -> list[str] | None:
        ''' translate a given enverb into system call(s) with hardcoded rule '''
        entityType = entityType.upper()
        if entityType.startswith('PROC'):
            entityType = 'PROC'
        if entityType.startswith('NET'):
            entityType = 'NET'
        if entityType.startswith('INFO'):
            entityType = 'INFO'
        entityType_lst = ['FILE', 'PROC', 'DEVICE', 'INFO', 'NET']
        if entityType not in entityType_lst:
            return None
        
        # Get rules belongs to this entityType
        dataFileType = self.df[self.df['EntityType'] == entityType]
        dataEnVerb = dataFileType.loc[dataFileType['EnVerb'].isin([enVerb])]
        if len(dataEnVerb) == 0:
            return None
        return list(dataEnVerb['Syscall'].unique())

    def resolve(self, entityType: str, syscall:str, en_verb:str, downcast=False) -> bool:
        hit, isSysChange = False, False
        corresponding_syscall_list = self.translate_rule(entityType, en_verb)
        if corresponding_syscall_list and syscall in corresponding_syscall_list:
            hit = True
        elif entityType.upper() == "FILE" and syscall in self.update_syscall_list:
            isSysChange = True
        return hit, isSysChange

# test
oE = OperationEvaluator()
oE.translate_rule('FILE', 'read') # ['read', 'readlink']
ans_syscalls = oE.translate_rule('FILE', 'change') # ['rename', 'fchmod', 'chmod', 'lchmod']
print(ans_syscalls)
ans_syscalls = oE.translate_rule('PROC', 'change') # ['ptrace', 'brk']
print(ans_syscalls)
ans_syscalls = oE.translate_rule('FILE', 'create') # ['dup', 'dup2', 'open', 'mmap', 'link']
print(ans_syscalls)
ans_syscalls = oE.translate_rule('PROC', 'create') # ['clone', 'fork', 'vfork']
print(ans_syscalls)
ans_syscalls = oE.translate_rule('NET', 'create') # ['pipe', 'socket']
print(ans_syscalls)

# test
res = oE.resolve('FILE','linkat', 'read') # (False, False)
print(res) 

In [ ]:
# ! pip install transformers

In [ ]:
import torch
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
% matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load pre-trained BERT model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()


In [ ]:
# Run the text through BERT, and collect all of the hidden states produced
# from all 12 layers. 
with torch.no_grad():

    outputs = model(tokens_tensor, segments_tensors)

    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]
